In [ ]:
import numpy as np
import sys
import os
from keras.models import Model
from keras import backend as K
from keras.regularizers import l1
from keras.utils import plot_model
from keras.datasets import mnist, cifar10
from keras.layers import Lambda, Input, Dense, Flatten, Reshape, Dropout, BatchNormalization
from keras.losses import mse, binary_crossentropy, kullback_leibler_divergence
from keras.layers.convolutional import Conv1D, Conv2DTranspose, UpSampling1D, MaxPooling1D
import h5py
from ml4cvd.arguments import parse_args

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import keras.optimizers as optimizers
from ml4cvd.tensor_generators import test_train_valid_tensor_generators

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
# network parameters
input_shape = (64, 1)
dropout_rate = 0.1
latent_dim = 8

In [ ]:
# VAE model = encoder + decoder
# build encoder model

inputs = Input(shape=input_shape, name='input_trend_heartrate_ecg_bike')
x = Conv1D(256, 3, strides=2, activation='relu', padding='same')(inputs)
x = Dropout(dropout_rate)(x)
x = Conv1D(128, 3, strides=2, activation='relu', padding='same')(x)
x = Dropout(dropout_rate)(x)
x = Conv1D(96, 3, activation='relu', padding='same')(x)
x = Dropout(dropout_rate)(x)
x = Flatten()(x)
x = Dense(latent_dim)(x)

z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

# build decoder model
inner_shape = (latent_dim,)
latent_inputs = Input(shape=inner_shape, name='z')
x = Dense(1536)(latent_inputs)
x = Reshape((16, 96))(x)
x = Conv1D(96, 3, strides=1, activation='relu', padding='same')(x)
x = UpSampling1D()(x)
x = Conv1D(128, 3, strides=1, activation='relu', padding='same')(x)
x = UpSampling1D()(x)
x = Conv1D(160, 3, activation='relu', padding='same')(x)
outputs = Conv1D(1, 1, activation='linear')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='output_trend_heartrate_ecg_bike')
decoder.summary()

In [ ]:
epochs = 10
batch_size = 128
sys.argv = ['train', 
            '--tensors','/mnt/disks/ecg-bike-tensors/2019-10-10/', 
            '--input_tensors', 'ecg-bike-interp-hrs',
            '--output_tensors', 'ecg-bike-interp-hrs',
            '--batch_size', str(batch_size), 
            '--epochs', str(epochs),
           ]
args = parse_args()
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(args.tensor_maps_in,  args.tensor_maps_out,  args.tensors, args.batch_size,   args.valid_ratio, args.test_ratio, args.test_modulo, args.balance_csvs)

In [ ]:
a = generate_train.__next__()

In [ ]:
a[0].keys(), a[1].keys()

In [ ]:
outputs = decoder(encoder(inputs)[2])  # 2 is the sampled z
vae = Model(inputs, outputs, name='vae_cnn')
vae.summary()

# Compute VAE loss
def vae_loss(y_true, y_pred):
    reconstruction_loss = mse(y_true, y_pred)
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(10 * K.mean(reconstruction_loss) + kl_loss)
    return vae_loss


vae.compile(optimizer=optimizers.Adam(1e-3), loss=vae_loss)
vae.summary()
vae.fit_generator(generate_train, epochs=epochs, steps_per_epoch=128, validation_steps=32, validation_data=generate_valid)

In [ ]:
x_test_encoded = encoder.predict_generator(generate_test, steps=10)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[2][:, 0], x_test_encoded[2][:, 1], cmap='jet')
plt.show()

In [ ]:
x_test = generate_test.__next__()
x_test = x_test[0]['input_trend_heartrate_ecg_bike']

In [ ]:
for i in np.random.randint(0, len(x_test), 5):
    plt.figure(figsize=(10, 5))
    recon = vae.predict(x_test[i: i+1])
    plt.plot(recon[0], c='r', label='Reconstruction')
    plt.plot(x_test[i], c='k', linestyle='--', label='Actual')
    plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
low, hi = -3, 3
for m in np.linspace(low, hi, 50):
    inp = np.zeros((1, latent_dim))
    c = (m - low) / (hi - low)
    lw = (1-c) * 1 + 1
    c = (c, .2, .5)
    inp[0, 0] = m
    
    recon = decoder.predict(inp)
    plt.plot(recon[0], label=f'{m:.2f}', c=c, linewidth=lw)
plt.show()
plt.figure(figsize=(15, 8))
low, hi = -1, 1
for m in np.linspace(low, hi, 50):
    inp = np.zeros((1, latent_dim))
    c = (m - low) / (hi - low)
    lw = (1-c) * 1 + 1
    c = (c, .2, .5)
    inp[0, 1] = m
    
    recon = decoder.predict(inp)
    plt.plot(recon[0], label=f'{m:.2f}', c=c, linewidth=lw)

In [ ]:
plt.figure(figsize=(15, 8))

low, hi = 0, 5
raw = x_test[0:1]

inp = encoder.predict(raw)[0]

r = decoder.predict(inp)
for m in np.linspace(low, hi, 50):
    c = (m - low) / (hi - low)
    lw = (1-c) * 1 + 1
    c = (c, .2, .5)
    inp = encoder.predict(raw)[0]
    inp[0, 7] += m 
    recon = decoder.predict(inp)
    plt.plot(recon[0] - r[0], label=f'{m:.2f}', c=c, linewidth=lw)
plt.plot(r[0], c='k', linestyle='--')